In [ ]:
# df1 = pd.read_csv('resume_data_improved.csv')
# df2 = pd.read_csv('resume_data_developer.csv')
# pattern = "resume_data"
# files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.startswith(pattern)]
# df_full = [pd.read_csv(csv) for csv in files]
# df = pd.concat(df_full, axis=0, ignore_index=True).drop_duplicates()
# df.shape

# START FROM HERE

In [3]:
import re, numpy as np
import os, pandas as pd
directory = os.path.join(os.getcwd(), 'preprocess')
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and re.search(r'category_[0-9]+', f)]
files

['resume_data_category_35',
 'resume_data_category_84',
 'resume_data_category_46',
 'resume_data_category_40',
 'resume_data_category_8',
 'resume_data_category_22',
 'resume_data_category_10',
 'resume_data_category_87',
 'resume_data_category_121',
 'resume_data_category_14',
 'resume_data_category_23',
 'resume_data_category_1',
 'resume_data_category_122']

In [17]:
df2 = pd.concat([pd.read_csv(f'{directory}/{csv}') for csv in files], axis=0, ignore_index=True).drop_duplicates()

In [18]:
df2['Desired Industry'].value_counts()

Engineering                       1500
Information Technology            1500
Accounting/Bookkeeping            1500
Business/Management               1500
Finance/Investment                1276
Computer Software/Programming     1057
Banking/Mortgage                  1007
Graphics Design/Commercial Art     604
CAD/Drafting                       556
Automotive/Mechanic                483
Computer Hardware                  330
Quality Control                    289
Public Relations                   227
Name: Desired Industry, dtype: int64

In [19]:
df2 = df2.drop(['Availability Date', 'Date Posted', 'Unnamed: 0', 'Reference', 'Affiliations', 'Additional Information'], axis=1)

In [20]:
df2.shape

(11829, 15)

In [21]:

attr = ['Skills', 'Objective', 'Education', 'Experience']
for col in attr:
    df2[col] = df2[col].apply(lambda x: str(x).encode('ascii', errors='ignore').decode().strip())
    df2[col] = df2[col].replace(r'^[-=]+[-=]$', np.nan ,  regex=True)
    df2[col] = df2[col].replace(['nan', 'non'], np.nan)
    df2[col] = df2[col].str.replace('\t', '')
    df2[col] = df2[col].str.replace('\n', ' ')

In [22]:
df2.columns

Index(['Role', 'Desired Industry', 'SpiderID', 'Desired Job Location',
       'Type of Position', 'Desired Wage', 'U.S. Work Authorization',
       'Job Level', 'Willing to Travel', 'Highest Degree Attained',
       'Willing to Relocate', 'Objective', 'Experience', 'Education',
       'Skills'],
      dtype='object')

In [23]:
df2.shape

(11829, 15)

In [24]:
df2 = df2.fillna('')

In [13]:
# df2.to_csv('data.csv')

In [14]:
# df2['Skills'] + ' '+ df2['Objective']

In [37]:
tot = (df2 == '').sum()

# **NLP PART**

In [12]:
import spacy
spacy.prefer_gpu()

False

In [13]:
nlp = spacy.load('en_core_web_sm')

In [27]:
df_copy = df2.copy()

In [31]:
iteration = 0
def preprocess_doc(doc):
    # Tokenize the document
    # tokens = [token.text for token in doc]

    # Remove stop words
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    # Join the tokens back into a single string
    preprocessed_doc = ' '.join(tokens)
    return preprocessed_doc

In [32]:
for col in ['Skills', 'Objective', 'Education', 'Experience']:
    df_copy[col] = df_copy[col].apply(lambda x: preprocess_doc(nlp(x)))

In [33]:
df_copy[['Skills', 'Objective', 'Education', 'Experience']]

,Skills,Objective,Education,Experience
0,Preventive Maintenance Process Improvement Cha...,skilled carry preventative corrective general ...,Bachelor Arts Geosciences University North C...,Metro Ports Wilmington NC Maintenance Manager ...
1,java,goal orient hardworke engineer graduate keen i...,Philippines Licensed Mechanical Engineer 15 ye...,versatile technically astute field piping qc t...
2,expert review mechanical site design drawing r...,hand analytical solution focus professional ex...,Bachelor Technology Mechanichal Engineering+ A...,Technical Management Trainee Eltex Appliances ...
3,Mechanichal Engineering + mechatronic,secure challenging role expand knowledge skill...,Bachelor Engineering Mechanical Engineering Ho...,hand analytical solution focus professional ex...
4,Leadership Role HKIE Student Chapters Committe...,TARGET Jobspider Senior Mems Process Engineer ...,hand analytical solution focus professional ex...,Gururaj Engineers automation Electrical Engine...
...,...,...,...,...
11824,,,,
11825,,,,
11826,,,,
11827,,,,


In [51]:
(df_copy[['Skills', 'Objective', 'Education', 'Experience']] == '').sum() - tot

Desired Industry            NaN
Desired Job Location        NaN
Desired Wage                NaN
Education                  19.0
Experience                 24.0
Highest Degree Attained     NaN
Job Level                   NaN
Objective                  15.0
Role                        NaN
Skills                     21.0
SpiderID                    NaN
Type of Position            NaN
U.S. Work Authorization     NaN
Willing to Relocate         NaN
Willing to Travel           NaN
dtype: float64

In [28]:
df_copy.shape

(11829, 15)

In [38]:
df_copy.to_csv('data.csv')

In [65]:
df_copy

,Role,Desired Industry,SpiderID,Desired Job Location,Type of Position,Desired Wage,U.S. Work Authorization,Job Level,Willing to Travel,Highest Degree Attained,Willing to Relocate,Objective,Experience,Education,Skills
0,Director of Maintenance Operations Resume,Engineering,83950,"Wilmington, North Carolina",Full-Time Permanent,115000,Yes,"Management (Manager, Director)","Yes, Less Than 25%",Bachelors,Yes,,,,
1,java developer Resume,Engineering,83927,"new york, Delaware",Contractor,,Yes,"Management (Manager, Director)","Yes, More Than 75%",Bachelors,Yes,,,,
2,Mechanical Engineer Resume,Engineering,83916,"Surrey, New Brunswick",Full-Time Permanent,,Yes,Experienced with over 2 years experience,,Bachelors,Yes,,,,
3,Technician Resume,Engineering,83864,"Edmonton, Alberta",Part-Time Permanent,22$/hour,,New Grad/Entry Level,"Yes, 50-75%",Bachelors,Yes,,,,
4,Mechanical Engineer Resume,Engineering,83803,"Calgary, Alberta",Full-Time Permanent,,No,New Grad/Entry Level,,Bachelors,Undecided,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11824,management Resume,Business/Management,26166,"Arlington, Virginia",Full-Time Permanent,50000,Yes,"Management (Manager, Director)","Yes, Less Than 25%",High School/Equivalent,No,,,,
11825,Office Administration Resume,Business/Management,26160,"Queen Creek, Arizona",Full-Time Permanent,"41,000",Yes,Experienced with over 2 years experience,,Bachelors,No,,,,
11826,"CEO, The Associated Consultants, LLC Resume",Business/Management,26142,"Gaithersburg, Maryland",Full-Time Permanent,,,"Executive (President, VP, CEO)",,Masters,,,,,
11827,"Sales, Account Manager, Business Development, ...",Business/Management,26118,"Winchester, Virginia",Full-Time Permanent,"65,000.00",Yes,"Management (Manager, Director)",,Bachelors,No,,,,


In [39]:
df = pd.read_csv('data/data.csv')
df = df.drop('Unnamed: 0', axis=1)

In [24]:
from sklearn.model_selection import train_test_split
feature = df_copy['Desired Industry'].values
X = df_copy.drop('Desired Industry', axis=1)

In [52]:
from spacy import displacy
def ner(doc):
    displacy.render(doc ,style='ent')
    print('\n')

In [35]:
df_copy.loc[n, ['Skills', 'Education']]

AttributeError: 'DataFrame' object has no attribute 'map'

In [54]:
for row in np.random.randint(0, 11829, 10):
    ner(nlp(df.loc[row, 'Skills']))

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'float'>

In [55]:
df_copy['Desired Industry'].

,Role,Desired Industry,SpiderID,Desired Job Location,Type of Position,Desired Wage,U.S. Work Authorization,Job Level,Willing to Travel,Highest Degree Attained,Willing to Relocate,Objective,Experience,Education,Skills
0,Director of Maintenance Operations Resume,Engineering,83950,"Wilmington, North Carolina",Full-Time Permanent,115000,Yes,"Management (Manager, Director)","Yes, Less Than 25%",Bachelors,Yes,skilled carry preventative corrective general ...,Metro Ports Wilmington NC Maintenance Manager ...,Bachelor Arts Geosciences University North C...,Preventive Maintenance Process Improvement Cha...
1,java developer Resume,Engineering,83927,"new york, Delaware",Contractor,,Yes,"Management (Manager, Director)","Yes, More Than 75%",Bachelors,Yes,goal orient hardworke engineer graduate keen i...,versatile technically astute field piping qc t...,Philippines Licensed Mechanical Engineer 15 ye...,java
2,Mechanical Engineer Resume,Engineering,83916,"Surrey, New Brunswick",Full-Time Permanent,,Yes,Experienced with over 2 years experience,,Bachelors,Yes,hand analytical solution focus professional ex...,Technical Management Trainee Eltex Appliances ...,Bachelor Technology Mechanichal Engineering+ A...,expert review mechanical site design drawing r...
3,Technician Resume,Engineering,83864,"Edmonton, Alberta",Part-Time Permanent,22$/hour,,New Grad/Entry Level,"Yes, 50-75%",Bachelors,Yes,secure challenging role expand knowledge skill...,hand analytical solution focus professional ex...,Bachelor Engineering Mechanical Engineering Ho...,Mechanichal Engineering + mechatronic
4,Mechanical Engineer Resume,Engineering,83803,"Calgary, Alberta",Full-Time Permanent,,No,New Grad/Entry Level,,Bachelors,Undecided,TARGET Jobspider Senior Mems Process Engineer ...,Gururaj Engineers automation Electrical Engine...,hand analytical solution focus professional ex...,Leadership Role HKIE Student Chapters Committe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11824,management Resume,Business/Management,26166,"Arlington, Virginia",Full-Time Permanent,50000,Yes,"Management (Manager, Director)","Yes, Less Than 25%",High School/Equivalent,No,,,,
11825,Office Administration Resume,Business/Management,26160,"Queen Creek, Arizona",Full-Time Permanent,"41,000",Yes,Experienced with over 2 years experience,,Bachelors,No,,,,
11826,"CEO, The Associated Consultants, LLC Resume",Business/Management,26142,"Gaithersburg, Maryland",Full-Time Permanent,,,"Executive (President, VP, CEO)",,Masters,,,,,
11827,"Sales, Account Manager, Business Development, ...",Business/Management,26118,"Winchester, Virginia",Full-Time Permanent,"65,000.00",Yes,"Management (Manager, Director)",,Bachelors,No,,,,


In [58]:
df['Role'].nunique(

7316

In [60]:
df = df.drop(['SpiderID', 'U.S. Work Authorization', 'Desired Job Location', 'Willing to Travel'], axis=1)

In [61]:
df.shape

(11829, 11)

In [62]:
df.to_csv('preprocess/final_data.csv')

In [66]:
df['Desired Wage']

0           115000
1              NaN
2              NaN
3         22$/hour
4              NaN
           ...    
11824        50000
11825       41,000
11826          NaN
11827    65,000.00
11828          NaN
Name: Desired Wage, Length: 11829, dtype: object